# Notebook to Segment Skeletal Muscle Area from MRI images (TIFs)

###### Load libraries and directories

In [3]:
import os
os.getcwd()
os.chdir('/tf/smipipeline')
print(os.getcwd())

/tf/smipipeline


In [4]:
from IPython.display import display, HTML

In [5]:
# from IPython import get_ipython
from tqdm.notebook import tqdm
import pickle
import pprint
pp = pprint.PrettyPrinter(indent=1)

# Custom modules for debugging
from SliceViewer import ImageSliceViewer3D, ImageSliceViewer3D_1view,ImageSliceViewer3D_2views
from investigate import *

#pd.set_option("display.max_rows", 10)
      
import json
from run_sma_experiment import find_l3_images,output_images
import pprint
from L3_finder import *

# Custom functions
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

Using TensorFlow backend.


In [6]:
get_ipython().run_line_magic('tb', '')

ModuleNotFoundError: No module named 'SliceViewer'

In [7]:
cwd = os.getcwd()
print(cwd)
data = '/tf/data'
pickles = '/tf/pickles'
models = '/tf/models'

/tf/smipipeline


In [10]:
# Import modules and config file
configfile = os.path.join(cwd,'config/mri/run_prediction_CV_mri_tif_inverted.json')
with open(configfile, "r") as f:
        config = json.load(f)
pp.pprint(config)

{'l3_finder': {'cache_dir': '/tf/_cache/',
               'cache_intermediate_results': True,
               'model_path_dir': '/tf/models/l3/cv_final',
               'new_tim_dicom_dir_structure': True,
               'output_directory': '/tf/output/cv_poorl3/l3',
               'overwrite': True,
               'save_plots': True,
               'show_plots': False,
               'tif_dir': '/tf/data/sarcopeniaMR_L3_TIF'},
 'muscle_segmentor': {'model_path_dir': '/tf/models/muscle/cv_final',
                      'output_directory': '/tf/output/mri_ms_tif_inverted'}}


In [11]:
import csv
import sys
from collections import defaultdict
from pathlib import Path

from L3_finder import L3Image
from l3finder import ingest

from compare_best_to_manual_l3_and_seg import MinimalPrediction, MinimalResult

import SimpleITK as sitk
import multiprocessing
import run_sma_experiment
from myshow import myshow

### Segment L3 Axial Images and Calculate Muscle Area

In [14]:
# Load Image statistics dataframe for TIFs to calculate area
csvdir = '/tf/data'
csvname = [f for f in os.listdir(csvdir) if '.csv' in f][0]
infile  = os.path.join(csvdir,csvname)
df_imstats = pd.read_csv(infile, index_col=False)
display(df_imstats.head(2))

,pats,slice_no,orig_size,orig_spacing,orig_pixelarea,new_size,new_spacing,new_pixelarea
0,exam 9b,36,512,0.7813,0.610430,256,1.5626,2.441719
1,exam 99,10,512,0.6641,0.441029,256,1.3282,1.764115


In [15]:
from compare_best_to_manual_l3_and_seg import seg_model_configs

In [16]:
def normalize_image(image):
    image = sitk.GetArrayFromImage(image)
    mean = image.mean()
    std = image.std()
    #print("orig im mean: ", mean,"orig im std: ", std)
    norm_im = (image - mean) / std
    #print("norm im mean: ", norm_im.mean(),"norm im std: ", norm_im.std())
    return sitk.GetImageFromArray(norm_im)

def normalize_images(images):
    mean = images.mean()
    std = images.std()
    return (images - mean) / std

# v1: def invertimage_excluding_background(MR_nda,l_thresh=-0.5,h_thresh=1.7,seed=[128, 128]):
#v2 def invertimage_excluding_background(MR_nda,l_thresh_p=5,h_thresh_p=32,seed=[128, 128]): 
def invertimage_excluding_background(MR_nda,l_thresh_p=5,h_thresh_p=85,seed=[128, 128]): #v3
    MR = normalize_image(MR_nda)
    
    #** Only for V2 and above
    MR_range = max(MR) - min(MR)
#     print('MR min', min(MR))
#     print('MR max', max(MR))

    l_thresh = MR_range*l_thresh_p/100 + min(MR)
    h_thresh = MR_range*h_thresh_p/100 + min(MR)
#     print("new l_thresh: ", l_thresh)
#     print("new h_thresh: ", h_thresh)
    #**
    
    #
    # Blur using CurvatureFlowImageFilter
    #
    blurFilter = sitk.CurvatureFlowImageFilter()
    blurFilter.SetNumberOfIterations(5)
    blurFilter.SetTimeStep(0.125)
    image = blurFilter.Execute(MR)

    #
    # Set up ConneMRedThresholdImageFilter for segmentation
    #
    segmentationFilter = sitk.ConnectedThresholdImageFilter()
    segmentationFilter.SetLower(float(l_thresh))
    segmentationFilter.SetUpper(float(h_thresh))
    segmentationFilter.SetReplaceValue(1)
    segmentationFilter.AddSeed(seed)
    segmentationFilter.AddSeed([64,128]) # Adding additional seeds for safety
    segmentationFilter.AddSeed([128,64])
    # Run the segmentation filter
    image = segmentationFilter.Execute(image)
    image[seed] = 1
    

    # Fill holes
    image = sitk.BinaryFillhole(image);

    # Intensity Invert FIlter
    invertfilter = sitk.InvertIntensityImageFilter()
    MR_invert = normalize_image(invertfilter.Execute(MR))
#     print("MR_invert min: ", min(MR_invert))
#     print("MR_invert max: ", max(MR_invert))
    
    # Masking FIlter
    maskingFilter = sitk.MaskImageFilter()
    background_value = min(MR_invert)-0.5
    maskingFilter.SetOutsideValue(background_value)
#     print("Name: ", maskingFilter.GetName())
#     print("Masking Value: ", maskingFilter.GetMaskingValue())
#     print("Outside Value: ", maskingFilter.GetOutsideValue())
    MR_noTable = maskingFilter.Execute(MR_invert,image)
#     print("MR_noTable min: ", min(MR_noTable))
    return MR_noTable, image

In [17]:
def _load_l3_tif(imagepath):
    return sitk.GetArrayFromImage(sitk.ReadImage(imagepath))

def _load_l3_tif_invert(imagepath):
    img = sitk.ReadImage(imagepath)
    desired_img, mask = invertimage_excluding_background(img)
    return sitk.GetArrayFromImage(desired_img) #, mask, img

In [18]:
# DEBUG ONLY
debugpath = "/tf/data/sarcopeniaMR_L3_TIF/exam 97b/MR.tif"
# debugpath = "/tf/data/sarcopeniaMR_L3_TIF/exam 116/MR.tif"
#debugpath = "/tf/data/sarcopeniaMR_L3_TIF/exam 83/MR.tif"
#debugpath = "/tf/data/sarcopeniaMR_L3_TIF/exam 9b/MR.tif"
#debugpath = "/tf/data/sarcopeniaMR_L3_TIF/exam 202/MR.tif"
#debugpath = "/tf/data/sarcopeniaMR_L3_TIF/exam 145/MR.tif"
# image, mask, orig = _load_l3_tif_invert(debugpath)
# myshow(sitk.GetImageFromArray(image))
# myshow(mask)
# myshow(orig)

In [21]:
from skimage import io

In [22]:
sitkimg = _load_l3_tif(debugpath)

In [26]:
pilimg = io.imread(debugpath,plugin='pil')

In [32]:
type(sitkimg)

numpy.ndarray

In [332]:
def load_l3_tifs(input_dir, tifname='MR.tif', invert = False):
    print('Loading tifs for all patients in directory: ', input_dir)
    infiles = []
    patnames = []
    for root, directories, files in os.walk(input_dir):
        for folder in directories:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root,folder,tifname)
            if  os.path.isfile(filepath):
                infiles.append(filepath)
                patnames.append(folder)
            else:
                print('input file not found in ', folder)
            
    print('Number of patient images found: ', len(infiles))
    print('Loading images in parallel into numpy arrays')
    
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        if invert:
            ndas = list(tqdm(pool.imap(_load_l3_tif_invert, infiles)))
        else:
            ndas = list(tqdm(pool.imap(_load_l3_tif, infiles)))
        pool.close()
        pool.join()
        print('Parallel loading complete,list of arrays of length ', len(ndas), ' is returned')
        return ndas, patnames
    
def calculate_maskarea(pixelarea,mask):
    muscle_pixels = np.count_nonzero(mask == 1)
    return muscle_pixels*pixelarea

def get_manual_maskarea(input_dir, df_imstats, tifname='MASK.tif'):
    manual_smas = []
    infiles = []
    patnames = []
    for root, directories, files in os.walk(input_dir):
        for folder in directories:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root,folder,tifname)
            if  os.path.isfile(filepath):
                infiles.append(filepath)
                patnames.append(folder)
            else:
                print('input file not found in ', folder)
            
    print('Number of patient mask images found: ', len(infiles))
    print('Loading masks in parallel into numpy arrays')
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        masks = list(tqdm(pool.imap(_load_l3_tif, infiles)))
        pool.close()
        pool.join()
        print('Parallel loading complete,list of arrays of length ', len(masks), ' is returned')
    
    for mask,patname in zip(masks,patnames):
        muscle_pixels = np.count_nonzero(mask == 1)
        pixelarea = df_imstats.loc[df_imstats['pats']==patname,'new_pixelarea'].values[0]
        manual_smas.append(muscle_pixels*pixelarea)
    
    return manual_smas,masks

In [333]:
def do_segmentation_cv_mri_tifs(seg_model_config,l3_ndas, patnames, df_imstats):
    l3_ndas = np.array(l3_ndas)
    print("- Normalizing images")
    normalized_images = run_sma_experiment.normalize_images(l3_ndas)
    print("- Resizing images")
    resized_images = run_sma_experiment.resize_images(normalized_images)
    xs = run_sma_experiment.reshape_for_model(resized_images)
    reshaped_masks_list = []
    print(' Segmenting using all model configs')
    for config in seg_model_config:
        print("segmenting for model", config)
        model = run_sma_experiment.configure_and_load_model(config["model_path"])
        ys = model.predict(xs)
        masks = run_sma_experiment.convert_to_images(ys)        
        reshaped_masks = run_sma_experiment.reshape_masks(masks)
        reshaped_masks_list.append(reshaped_masks)
    average_masks = []
    print("Finding Average Masks")
    for one_subjects_masks in zip(*reshaped_masks_list):
        sitk_images = [sitk.Cast(sitk.GetImageFromArray(mask), sitk.sitkUInt8)
                       for mask in one_subjects_masks]
        result = sitk.LabelVoting(sitk_images, 1)
        arr = sitk.GetArrayFromImage(result)
        average_masks.append(arr)
    print("Finding sma")
    smas = []
    for patname, mask in zip(patnames,average_masks):
        smas.append(calculate_maskarea(df_imstats.loc[df_imstats['pats']==patname,'new_pixelarea'].values[0], mask))
    return smas,average_masks

In [334]:
if __name__ == "__main__":
    l3_ndas, patnames = load_l3_tifs(config["l3_finder"]["tif_dir"], invert=True)
    configs = seg_model_configs(config["muscle_segmentor"]['model_path_dir'])
    smas, average_masks = do_segmentation_cv_mri_tifs(configs, l3_ndas, patnames, df_imstats)
    manual_smas, manual_masks = get_manual_maskarea(config["l3_finder"]["tif_dir"], df_imstats)
    print('Length of smas normals: ',len(smas))
    print('Length of average_masks normals: ',len(average_masks))
    print("Done")

Loading tifs for all patients in directory:  /tf/data/sarcopeniaMR_L3_TIF
Number of patient images found:  201
Loading images in parallel into numpy arrays



Parallel loading complete,list of arrays of length  201  is returned
- Normalizing images
- Resizing images
 Segmenting using all model configs
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_0.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_1.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_2.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_3.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_4.h5'}
Finding Average Masks
Finding sma
Number of patient mask images found:  201
Loading masks in parallel into numpy arrays



Parallel loading complete,list of arrays of length  201  is returned
Length of smas normals:  201
Length of average_masks normals:  201
Done


In [335]:
from imageio import imsave
import csv

def output_sma_results_tifs(output_dir, mrs, average_masks, smas, manual_masks, manual_smas, patnames):
    os.makedirs(output_dir, exist_ok=True)
    csv_filename = os.path.join(output_dir, "areas_by_subject_id.csv")
    with open(csv_filename, "w") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["subject_id", "area", "manual_area", "relative_diff"])
        print('Saving Segmentation Results in ', output_dir)
        index = 0    
        for mask, mr, sma, manual_mask, manual_sma,pat in zip(average_masks, mrs, smas, manual_masks, manual_smas, patnames):
            index += 1
            base = os.path.join(output_dir, str(index) + "_" + pat)
            imsave(base + "_MR.tif", mr.astype(np.float32))
            imsave(base + "_muscle.tif", mask * np.iinfo(np.uint8).max)
            imsave(base + "_truth.tif", manual_mask * np.iinfo(np.uint8).max)
            rel_diff = abs(manual_sma - sma)/manual_sma
            row = [
                pat,
                sma,
                manual_sma,
                rel_diff,
            ]
            csv_writer.writerow(row)
        print('Total exams outputted: ', index)

In [336]:
output_sma_results_tifs(config["muscle_segmentor"]['output_directory'], l3_ndas, average_masks, smas, manual_masks, manual_smas, patnames)

Saving Segmentation Results in  /tf/output/mri_ms_tif_inverted
Total exams outputted:  201
